In [1]:
import duckdb
import os
import pandas as pd
import urllib.request 
from zipfile import ZipFile 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


from matplotlib import pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
path_to_export = "/Users/yashwanth/Documents/GWU/Sem 3/Data Mining/Class 2/Class Material/MovieLensExport/" 
duckdb.sql(f'''IMPORT DATABASE '{path_to_export}' ''')
#Let's check that the data all loaded!
duckdb.sql('SHOW TABLES')

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ date_dim     │
│ driver       │
│ driver2      │
│ genre_dim    │
│ links        │
│ movie_dim    │
│ movies       │
│ ratings      │
│ ratings_fact │
│ tags         │
│ user_dim     │
│ user_matrix  │
├──────────────┤
│   12 rows    │
└──────────────┘

# Part 1: Dimensional Modeling
# Question 1
Question 1a: Data Quality is a very important part of the data ingestion process in a data warehouse. We did some parsing of the movie titles to get the "extracted" title and "year released". Can you find some examples where our parsing is wrong?

Question 1b: What do you think we could do to fix these kinds of errors? A qualitative description is fine.

In [4]:
# Question 1a 

movie_dim = duckdb.sql('SELECT * FROM movie_dim').df()

incorrect_year = movie_dim[movie_dim['year_released'].isna() | ~movie_dim['year_released'].str.match(r'^\d{4}$')]
incorrect_title = movie_dim[movie_dim['extracted_title'].str.len() < 1]

print("Rows with incorrect year released:")
print(incorrect_year.head())

print("Rows with incorrect extracted title:")
print(incorrect_title.head())

Rows with incorrect year released:
      movieId                                              title  \
588       889                                  1-900 (06) (1994)   
5750    40697                                          Babylon 5   
6862    69757                        (500) Days of Summer (2009)   
8831   140956                                   Ready Player One   
8926   147250  The Adventures of Sherlock Holmes and Doctor W...   

                                        extracted_title year_released  \
588                                        1-900 (1994)            06   
5750                                          Babylon 5                 
6862                               (500) Days of Summer           500   
8831                                   Ready Player One                 
8926  The Adventures of Sherlock Holmes and Doctor W...                 

       imdbId    tmdbId  
588   0109001  101230.0  
5750  0105946       NaN  
6862  1022603   19913.0  
8831  1677720

Question 1b 

To improve how we extract years from movie titles, we should start by refining our regular expressions to accurately capture the year. Better regex patterns can help ensure we’re getting the year right from the title. It’s also better to handle special cases where numbers in the title might be mistaken for a year. For example, a title like "The 300 Spartans" includes a number that isn’t a year. We should implement validation checks to make sure the extracted years are valid and fall within a sensible range. Lastly, cross-referencing the extracted data with external sources like IMDb can help us verify and correct the title and year information, ensuring our dataset is as accurate as possible.

# Question 2

Let's just write a few queries for getting the feel of using a schema like this.

Question 2a: Which genre has the most ratings?

Question 2b: Which genrea has the highest average rating?

In [7]:
# Question 2a

most_rated_genre = duckdb.sql('''
SELECT
    genre,
    COUNT(*) AS num_ratings
FROM
    ratings_fact r
JOIN
    genre_dim g
ON
    r.movieId = g.movieId
GROUP BY
    genre
ORDER BY
    num_ratings DESC
LIMIT 1
''').df()

most_rated_genre

,genre,num_ratings
0,Drama,41928


In [8]:
# Question 2b

highest_avg_rating_genre = duckdb.sql('''
SELECT
    genre,
    AVG(rating) AS avg_rating
FROM
    ratings_fact r
JOIN
    genre_dim g
ON
    r.movieId = g.movieId
GROUP BY
    genre
ORDER BY
    avg_rating DESC
LIMIT 1
''').df()

highest_avg_rating_genre

,genre,avg_rating
0,Film-Noir,3.920115


# Question 3

In our cleaning/curation of the data to "load" it into this format, it may have seemed a little overkill to create a date_dim table. But, let's compare what a query looks like with and without our processing.

Question 3a: Using the processed version of the tables ratings_fact and date_dim, write a query that counts how many Science Fiction movies were reviewed on Sundays.

Question 3b: Using the raw versions of the data, ratings and movies write a query to answer the same question?

Question 3c: Which query was easier to write? Which do you think is more approachable for a business analyst?

In [10]:
# Question 3a

sci_fi_sundays_processed = duckdb.sql('''
SELECT 
    COUNT(rf.rating) AS num_reviews
FROM 
    ratings_fact rf
JOIN 
    date_dim dd ON rf.rating_date = dd.date_key
JOIN 
    genre_dim gd ON rf.movieId = gd.movieId
WHERE 
    gd.genre = 'Sci-Fi' AND dd.day_name = 'Sunday'
GROUP BY 
    dd.day_name;
''').df()

sci_fi_sundays_processed


,num_reviews
0,2734


In [11]:
# Question 3b

sci_fi_sundays_raw = duckdb.sql('''
SELECT 
    COUNT(*) AS num_reviews
FROM 
    ratings r
JOIN 
    movies m ON r.movieId = m.movieId
JOIN 
    genre_dim g ON m.movieId = g.movieId
WHERE 
    g.genre = 'Sci-Fi'
AND EXTRACT(DOW FROM TO_TIMESTAMP(r.timestamp)) = 0;
''').df()

sci_fi_sundays_raw


,num_reviews
0,2851


Question 3c

Using the processed tables, such as `ratings_fact` and `date_dim`, made writing the query much easier because the date-related details were already pre-processed. This streamlined the query, reducing complexity and making it more straightforward to construct. For a business analyst, the processed tables are more approachable since they include pre-calculated fields like `day_name`. This makes the query more intuitive and simpler to understand, without requiring complicated data manipulations or in-depth knowledge of the underlying data structure.

# Part 2: Design Matrix, nulls, similarity

Movie-centric design matrix
Let's imagine that, instead of trying to analyze users and see how similar they are, or what groups they belong to, we want to do this with movies instead.

So, we want to create a design matrix that has movies as rows instead

# Question 4
Create an initial 'driver table' with the following fields:

movieId
number_reviews - the total number of times this movie has received a rating
average_review - the average review score of this movie

In [14]:
# Question 4

query_driver_table = '''
CREATE TABLE movie_driver AS
SELECT 
    m.movieId,
    COUNT(r.rating) AS number_reviews,
    AVG(r.rating) AS average_review
FROM 
    movies m
JOIN 
    ratings r ON m.movieId = r.movieId
GROUP BY 
    m.movieId;
'''

duckdb.sql(query_driver_table)
movie_driver = duckdb.sql('SELECT * FROM movie_driver LIMIT 10').df()
movie_driver

,movieId,number_reviews,average_review
0,235,70,3.678571
1,423,10,2.850000
2,457,190,3.992105
3,500,144,3.388889
4,1031,24,3.479167
5,1222,102,4.098039
6,1240,131,3.896947
7,1282,53,3.783019
8,1793,2,4.000000
9,1920,18,2.833333


# Question 5: 

Categorical variables
We want to compare movies based on their genres. Because genres are a categorical variable, rather than numerical, we need to create what are called "indicator" or "dummy" variables.

In the case of movie genres, we should create a column for each genre and the value for each movie should be a 0/1 boolean of whether that movie is in that category.

For example, perhase we have a column/feature called Horror_genre_feature Then, every movie that is a horror movie will have a value of 1 in that column. Movies that are not horror movies will have a 0

Question 5: Create a design matrix, based on the driver table you made in question 4 that adds 1 column per genre

In [16]:
# Question 5

query_genre_indicators = '''
CREATE TABLE genre_indicators AS
SELECT 
    m.movieId,
    MAX(CASE WHEN gd.genre = 'Action' THEN 1 ELSE 0 END) AS Action_genre_feature,
    MAX(CASE WHEN gd.genre = 'Adventure' THEN 1 ELSE 0 END) AS Adventure_genre_feature,
    MAX(CASE WHEN gd.genre = 'Animation' THEN 1 ELSE 0 END) AS Animation_genre_feature,
    MAX(CASE WHEN gd.genre = 'Children' THEN 1 ELSE 0 END) AS Children_genre_feature,
    MAX(CASE WHEN gd.genre = 'Comedy' THEN 1 ELSE 0 END) AS Comedy_genre_feature,
    MAX(CASE WHEN gd.genre = 'Crime' THEN 1 ELSE 0 END) AS Crime_genre_feature,
    MAX(CASE WHEN gd.genre = 'Documentary' THEN 1 ELSE 0 END) AS Documentary_genre_feature,
    MAX(CASE WHEN gd.genre = 'Drama' THEN 1 ELSE 0 END) AS Drama_genre_feature,
    MAX(CASE WHEN gd.genre = 'Fantasy' THEN 1 ELSE 0 END) AS Fantasy_genre_feature,
    MAX(CASE WHEN gd.genre = 'Film-Noir' THEN 1 ELSE 0 END) AS Film_Noir_genre_feature,
    MAX(CASE WHEN gd.genre = 'Horror' THEN 1 ELSE 0 END) AS Horror_genre_feature,
    MAX(CASE WHEN gd.genre = 'IMAX' THEN 1 ELSE 0 END) AS IMAX_genre_feature,
    MAX(CASE WHEN gd.genre = 'Musical' THEN 1 ELSE 0 END) AS Musical_genre_feature,
    MAX(CASE WHEN gd.genre = 'Mystery' THEN 1 ELSE 0 END) AS Mystery_genre_feature,
    MAX(CASE WHEN gd.genre = 'Romance' THEN 1 ELSE 0 END) AS Romance_genre_feature,
    MAX(CASE WHEN gd.genre = 'Sci-Fi' THEN 1 ELSE 0 END) AS Sci_Fi_genre_feature,
    MAX(CASE WHEN gd.genre = 'Thriller' THEN 1 ELSE 0 END) AS Thriller_genre_feature,
    MAX(CASE WHEN gd.genre = 'War' THEN 1 ELSE 0 END) AS War_genre_feature,
    MAX(CASE WHEN gd.genre = 'Western' THEN 1 ELSE 0 END) AS Western_genre_feature
FROM 
    genre_dim gd
JOIN 
    movies m ON gd.movieId = m.movieId
GROUP BY 
    m.movieId;
'''

duckdb.sql(query_genre_indicators)

query_design_matrix = '''
CREATE TABLE design_matrix AS
SELECT 
    dt.movieId,
    dt.number_reviews,
    dt.average_review,
    gi.Action_genre_feature,
    gi.Adventure_genre_feature,
    gi.Animation_genre_feature,
    gi.Children_genre_feature,
    gi.Comedy_genre_feature,
    gi.Crime_genre_feature,
    gi.Documentary_genre_feature,
    gi.Drama_genre_feature,
    gi.Fantasy_genre_feature,
    gi.Film_Noir_genre_feature,
    gi.Horror_genre_feature,
    gi.IMAX_genre_feature,
    gi.Musical_genre_feature,
    gi.Mystery_genre_feature,
    gi.Romance_genre_feature,
    gi.Sci_Fi_genre_feature,
    gi.Thriller_genre_feature,
    gi.War_genre_feature,
    gi.Western_genre_feature
FROM 
    movie_driver dt
LEFT JOIN 
    genre_indicators gi ON dt.movieId = gi.movieId;
'''

duckdb.sql(query_design_matrix)
result_design_matrix = duckdb.sql('SELECT * FROM design_matrix LIMIT 5').df()
print(result_design_matrix)


   movieId  number_reviews  average_review  Action_genre_feature  \
0      235              70        3.678571                     0   
1      423              10        2.850000                     1   
2      457             190        3.992105                     0   
3      500             144        3.388889                     0   
4     1031              24        3.479167                     0   

   Adventure_genre_feature  Animation_genre_feature  Children_genre_feature  \
0                        0                        0                       0   
1                        0                        0                       0   
2                        0                        0                       0   
3                        0                        0                       0   
4                        1                        0                       1   

   Comedy_genre_feature  Crime_genre_feature  Documentary_genre_feature  \
0                     1                  

# Question 6: 

Similarity of movies
We discussed cosine similarity in the context of our user analysis.

Question 6a: Find the 10 most similar movie to Iron Man 2 (2010) using the cosine similarity of their genre vectors

Because genres can also be thought of as a set of categories for each movie, we can also measure what is called the Jaccard Similarity

𝐽𝑎𝑐𝑐𝑎𝑟𝑑(𝑈,𝑉)=|𝑈∩𝑉||𝑈∪𝑉| 

Where, in our case, U is the set of categories for movie 1 and V is the set of categories for movie 2. This may yield slightly different ordering than cosine similarity. The numerator is actually the same in both cases, but the denominator is somewhat different

Question 6b: Find the 10 most similar movie to Iron Man 2 (2010) using the Jaccard similarity of their genre vectors

In [18]:
# Question 6a:

from sklearn.metrics.pairwise import cosine_similarity

query_iron_man = '''
SELECT * FROM design_matrix
WHERE movieId = (
    SELECT movieId FROM movies WHERE title = 'Iron Man 2 (2010)'
);
'''
iron_man_df = duckdb.sql(query_iron_man).df()
iron_man_vector = iron_man_df.iloc[0].values[3:] 

query_all_movies = '''
SELECT movieId, Action_genre_feature, Adventure_genre_feature, Animation_genre_feature,
       Children_genre_feature, Comedy_genre_feature, Crime_genre_feature, Documentary_genre_feature,
       Drama_genre_feature, Fantasy_genre_feature, Film_Noir_genre_feature, Horror_genre_feature,
       IMAX_genre_feature, Musical_genre_feature, Mystery_genre_feature, Romance_genre_feature,
       Sci_Fi_genre_feature, Thriller_genre_feature, War_genre_feature, Western_genre_feature
FROM design_matrix
WHERE movieId != (
    SELECT movieId FROM movies WHERE title = 'Iron Man 2 (2010)'
);
'''
all_movies_df = duckdb.sql(query_all_movies).df()
all_movie_ids = all_movies_df['movieId'].values
all_genre_vectors = all_movies_df.iloc[:, 1:].values  # Exclude movieId column

similarities = cosine_similarity([iron_man_vector], all_genre_vectors)[0]

similarity_df = pd.DataFrame({
    'movieId': all_movie_ids,
    'similarity': similarities
})

similarity_df = similarity_df.sort_values(by='similarity', ascending=False)
top_10_cosine = similarity_df.head(10)
print("Top 10 most similar movies to 'Iron Man 2 (2010)' using Cosine Similarity:")
print(top_10_cosine)

Top 10 most similar movies to 'Iron Man 2 (2010)' using Cosine Similarity:
      movieId  similarity
5459   101076    1.000000
7289    52722    1.000000
1928     6934    1.000000
7046     6365    1.000000
1665      780    0.894427
6056    93363    0.894427
3608    46530    0.894427
8457   110102    0.894427
3603     5349    0.894427
193      7569    0.894427


In [19]:
# Question 6b:

from sklearn.metrics import jaccard_score
jaccard_similarities = [
    jaccard_score(iron_man_vector, other_vector, average='binary')
    for other_vector in all_genre_vectors
]

jaccard_similarity_df = pd.DataFrame({
    'movieId': all_movie_ids,
    'jaccard_similarity': jaccard_similarities
})

jaccard_similarity_df = jaccard_similarity_df.sort_values(by='jaccard_similarity', ascending=False)
top_10_jaccard = jaccard_similarity_df.head(10)
print("Top 10 most similar movies to 'Iron Man 2 (2010)' using Jaccard Similarity:")
print(top_10_jaccard)

Top 10 most similar movies to 'Iron Man 2 (2010)' using Jaccard Similarity:
      movieId  jaccard_similarity
5459   101076                 1.0
7289    52722                 1.0
7046     6365                 1.0
1928     6934                 1.0
6062    94018                 0.8
4810    34048                 0.8
3665    53996                 0.8
2660    61248                 0.8
6983      480                 0.8
6056    93363                 0.8
